# 積分ステップ0.01で全てシミュレーションして、データを生成する

In [3]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme()
plt.rcParams["font.family"] = 'Hiragino sans'
sns.set(font=['Hiragino sans'])

In [2]:
def RungeKutta4(initial, time, model, F):
    """
    Parameters
    ----------
    F : 
        Forcing constant, variable used in L96
    """
    dt = time[1] - time[0]
    states = [initial]
    x = initial
    for t in time[:-1]:
        k1 = model(x, F)
        x1 = x + k1 * dt/2
        k2 = model(x1, F)
        x2 = x + k2 * dt/2
        k3 = model(x2, F)
        x3 = x + k3 * dt
        k4 = model(x3, F)
        x = x + (k1 + 2*k2 + 2*k3 + k4) * dt / 6
        states.append(x)
    states = np.stack(states)
    return states

def L96(x, F):
    """
    Lorenz 96 model with constant forcing.
    Cited by "https://en.wikipedia.org/wiki/Lorenz_96_model"
    
    Parameters
    ----------
    x : 
        variables
    F :
       Forcing constant 
    N : int
        number of sites
    """
    N = 40
    # Setting up vector
    d = np.zeros(N)
    # Loops over indices (with operations and Python underflow indexing handling edge cases)
    for i in range(N):
        d[i] = (x[(i + 1) % N] - x[i - 2]) * x[i - 1] - x[i] + F
    return d

## RK4シミュレーション

In [3]:
"""
観測地点: N=40
外力: F=8.0
外力変化率: ratio = 0.001
年数: 3年
time step: 積分間隔。0.2stepが１日　← 可変
total step: 計算式 - 0.2(/d) * 365(d/y) * year
"""
N = 40
F = 8.0
ratio = 0.001
year = 2
time_step = 0.01
total_step = 0.2 * 365 * year
time = np.arange(0.0, total_step + time_step, time_step, dtype=float)

x = np.full(N, F, dtype=float)
x[19] += F * ratio
dat = RungeKutta4(x, time, L96, F=F)

In [4]:
dat.shape

(14601, 40)

In [5]:
with open("RK4_GeneratedData.pkl", "wb") as f:
    pickle.dump(dat, f)

## noise項(pickle保存)

In [6]:
from numpy.random import Generator, MT19937

In [7]:
seed = 46
rg = Generator(MT19937(seed))
# ノイズとして加える正規分布
rand = rg.standard_normal(dat.shape)
print(dat.shape)

(14601, 40)


In [8]:
with open("NormalNoise.pkl", "wb") as f:
    pickle.dump(rand, f)

# test

In [1]:
def L96(x, F):
    """
    Lorenz 96 model with constant forcing.
    Cited by "https://en.wikipedia.org/wiki/Lorenz_96_model"
    
    Parameters
    ----------
    x : 
        variables
    N : int
        number of sites
    """
    # Setting up vector
    x_ = np.zeros(x.shape)
    N = len(x)
    # Loops over indices (with operations and Python underflow indexing handling edge cases)
    for i in range(N):
        x_[i] = (x[(i + 1) % N] - x[i - 2]) * x[i - 1] - x[i] + F
    return x_

In [4]:
a = L96(np.ones(40)*5, 8)
a

array([3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
       3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
       3., 3., 3., 3., 3., 3.])

In [5]:
x = np.ones(40) * 8
x[19] = x[19] * 1.001
x

array([8.   , 8.   , 8.   , 8.   , 8.   , 8.   , 8.   , 8.   , 8.   ,
       8.   , 8.   , 8.   , 8.   , 8.   , 8.   , 8.   , 8.   , 8.   ,
       8.   , 8.008, 8.   , 8.   , 8.   , 8.   , 8.   , 8.   , 8.   ,
       8.   , 8.   , 8.   , 8.   , 8.   , 8.   , 8.   , 8.   , 8.   ,
       8.   , 8.   , 8.   , 8.   ])

In [ ]:
def RK4(fun, step, F):
    """
    Parameters
    ----------
    F : 
        Forcing constant, variable used in L96
    """
    dt = step
    k1 = fun(x, F)
    x1 = x + k1 * dt/2
    k2 = fun(x1, F)
    x2 = x + k2 * dt/2
    k3 = fun(x2, F)
    x3 = x + k3 * dt
    k4 = fun(x3, F)
    x = x + (k1 + 2*k2 + 2*k3 + k4) * dt / 6